In [60]:
# Importer relevante biblioteker
import requests
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO

# PARAMETRE

lower = 4999
upper = 6000


# Definer en header, der bliver sendt med GET-request
header = {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36"
    }


# Sti til Arbejdstilsynets smiley-side
url = "https://websmiley.at.dk/websmiley/advancedsearchform.aspx"

# Send GET-request og hent Arbejdstilsynets smiley-side
r = requests.get(url,headers=header)

# Hvis status er OK parser scriptet indholdet, henter formData og tilføjer det til et dictionary
if r.status_code == 200:
    soup = BeautifulSoup(r.content, "html.parser")
    
    viewstate = soup.select("#__VIEWSTATE")[0]['value']
    eventvalidation = soup.select("#__EVENTVALIDATION")[0]['value']
    
    
    formData = {
        '__EVENTVALIDATION': eventvalidation,
        '__VIEWSTATE': viewstate,
        '__EVENTTARGET': '',
        'companyName': 'Virksomhedens navn',
        'cvrNr': 'CVR-nr.',
        'pNr': 'P-nr.',
        'ddlBrancheGrupper':0,
        'ddlRegioner': 0,
        'ddlSmiley':0,
        'btnDownload': 'Download smileydata'   
    }
    
    
# Send POST-request, hvor den netop hentede formdata sendes med
r = requests.post(url, data=formData, headers=header)



In [386]:
# Konverter responsdata til en string og indlæs i en DataFrame
# filtrer datasæt efter fynske virksomheder og røde smileyer
text = r.text
new = pd.read_csv(StringIO(text),sep=";",keep_default_na=False)
new = new[(new[" POSTNR"] > lower) & (new[" POSTNR"] < upper) & (new[" SMILEY"] == "rød")]
new[" SMILEY_DATO"] = pd.to_datetime(new[" SMILEY_DATO"])
new = new.sort_values(" SMILEY_DATO", ascending=False)

# Indlæs tidligere datasæt i en dataframe
# og filtrer
old = pd.read_csv("test.csv",sep=";",keep_default_na=False)
old = old[(old[" POSTNR"] > lower) & (old[" POSTNR"] < upper) & (old[" SMILEY"] == "rød")]
old[" SMILEY_DATO"] = pd.to_datetime(old[" SMILEY_DATO"])
old = old.sort_values(" SMILEY_DATO", ascending=False)

diff = pd.concat([new,old],keys=["new","old"]).drop_duplicates(keep=False)

if "new" in diff.index and "old" in diff.index:
    print("firma fjernet og firma tilføjet")
    print(diff[" NAVN"])

diff

,,CVRNR,PNR,BRANCHEGRUPPE,BRANCHE,NAVN,ADRESSE,POSTNR,POSTDISTRIKT,SMILEY,SMILEY_DATO,REAKTIONS_TYPE,ARBEJDSMILJØPROBLEMER,REAKTIONS_STATUS,Unnamed: 13
old,38,43223136,1027956196,Opførelse og nedrivning af byggeri,Nedrivning,RLH Entreprenør ApS,Toftegårdsvej 15,5620,Glamsbjerg,rød,2022-12-25,Strakspåbud,071 Kræftfremkaldende belastninger,Efterkommet,
